In [1]:
!pip -qqq install pip --progress-bar off
!pip -qqq install langchain-groq==0.1.3 --progress-bar off
!pip -qqq install langchain==0.1.17 --progress-bar off
!pip -qqq install llama-parse==0.1.3 --progress-bar off
!pip -qqq install qdrant-client==1.9.1  --progress-bar off
!pip -qqq install "unstructured[md]"==0.13.6 --progress-bar off
!pip -qqq install fastembed==0.2.7 --progress-bar off
!pip -qqq install flashrank==0.2.4 --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.17 requires langchain-core<0.4.0,>=0.3.33, but you have langchain-core 0.1.53 which is incompatible.
langchain-text-splitters 0.3.5 requires langchain-core<0.4.0,>=0.3.29, but you have langchain-core 0.1.53 which is incompatible.
  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
accelerate 1.3.0 requires huggingface-hub>=0.21.0, but you have huggingface-hub 0.20.3 which is incompatible.
peft 0.14.0 requires huggingface-hub>=0.25.0, but you have huggingface-hub 0.20.3 which is incompatible.
transformers 4.48.2 requires huggingface-hub<1.0,>=0.24.0, but you have huggingface-hub 0.20.3 which is incompatible.
transformers 4.48.2 requires tok

In [ ]:
import os

os.environ["GROQ_API_KEY"] = input("Enter your GROQ API Key: ")

In [4]:
import textwrap
from pathlib import Path

from google.colab import userdata
from IPython.display import Markdown
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Qdrant
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from llama_parse import LlamaParse

#Download and Parse the PDF Document

In [5]:
!mkdir data
!gdown 1ee-BhQiH-S9a2IkHiFbJz9eX_SfcZ5m9 -O "data/meta-earnings.pdf"

Downloading...
From: https://drive.google.com/uc?id=1ee-BhQiH-S9a2IkHiFbJz9eX_SfcZ5m9
To: /content/data/meta-earnings.pdf
100% 160k/160k [00:00<00:00, 84.6MB/s]


In [7]:
instruction = """The provided document is Meta First Quarter 2024 Results.
This form provides detailed financial information about the company's performance for a specific quarter.
It includes unaudited financial statements, management discussion and analysis, and other relevant disclosures required by the SEC.
It contains many tables.
Try to be precise while answering the questions"""

parser = LlamaParse(
    api_key="Your Llama Parse Key",
    result_type="markdown",
    parsing_instruction=instruction,
    max_timeout=5000,
)

llama_parse_documents = await parser.aload_data("./data/meta-earnings.pdf")

Started parsing the file under job_id 12ab26a2-92d2-466d-ab67-d63bf10a37ab


In [8]:
parsed_doc = llama_parse_documents[0]

In [9]:
Markdown(parsed_doc.text[:4096])

# Meta Reports First Quarter 2024 Results

MENLO PARK, Calif. – April 24, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter ended March 31, 2024.

"It's been a good start to the year," said Mark Zuckerberg, Meta founder and CEO. "The new version of Meta AI with Llama 3 is another step towards building the world's leading AI. We're seeing healthy growth across our apps and we continue making steady progress building the metaverse as well."

# First Quarter 2024 Financial Highlights

|In millions, except percentages and per share amounts|2024|2023|% Change|
|---|---|---|---|
|Revenue|$ 36,455|$ 28,645|27 %|
|Costs and expenses|$ 22,637|$ 21,418|6 %|
|Income from operations|$ 13,818|$ 7,227|91 %|
|Operating margin|38 %|25 %| |
|Provision for income taxes|$ 1,814|$ 1,598|14 %|
|Effective tax rate|13 %|22 %| |
|Net income|$ 12,369|$ 5,709|117 %|
|Diluted earnings per share (EPS)|$ 4.71|$ 2.20|114 %|

# First Quarter 2024 Operational and Other Financial Highlights

- Family daily active people (DAP) – DAP was 3.24 billion on average for March 2024, an increase of 7% year-over-year.
- Ad impressions – Ad impressions delivered across our Family of Apps increased by 20% year-over-year.
- Average price per ad – Average price per ad increased by 6% year-over-year.
- Revenue – Total revenue and revenue on a constant currency basis were $36.46 billion and $36.35 billion, respectively, both of which increased by 27% year-over-year.
- Costs and expenses – Total costs and expenses were $22.64 billion, an increase of 6% year-over-year.
- Capital expenditures – Capital expenditures, including principal payments on finance leases, were $6.72 billion.
- Capital return program – Share repurchases were $14.64 billion of our Class A common stock and dividends payments were $1.27 billion.
- Cash, cash equivalents, and marketable securities – Cash, cash equivalents, and marketable securities were $58.12 billion as of March 31, 2024. Free cash flow was $12.53 billion.
- Headcount – Headcount was 69,329 as of March 31, 2024, a decrease of 10% year-over-year.
---
# CFO Outlook Commentary

We expect second quarter 2024 total revenue to be in the range of $36.5-39 billion. Our guidance assumes foreign currency is a 1% headwind to year-over-year total revenue growth, based on current exchange rates.

We expect full-year 2024 total expenses to be in the range of $96-99 billion, updated from our prior outlook of $94-99 billion due to higher infrastructure and legal costs. For Reality Labs, we continue to expect operating losses to increase meaningfully year-over-year due to our ongoing product development efforts and our investments to further scale our ecosystem.

We anticipate our full-year 2024 capital expenditures will be in the range of $35-40 billion, increased from our prior range of $30-37 billion as we continue to accelerate our infrastructure investments to support our artificial intelligence (AI) roadmap. While we are not providing guidance for years beyond 2024, we expect capital expenditures will continue to increase next year as we invest aggressively to support our ambitious AI research and product development efforts.

Absent any changes to our tax landscape, we expect our full-year 2024 tax rate to be in the mid-teens.

In addition, we continue to monitor an active regulatory landscape, including the increasing legal and regulatory headwinds in the EU and the U.S. that could significantly impact our business and our financial results.

Q1 was a good start to the year. We're seeing strong momentum within our Family of Apps and are making important progress on our longer-term AI and Reality Labs initiatives that have the potential to transform the way people interact with our services over the coming years.
---
# Webcast and Conference Call Information

Meta will host a conference call to discuss the results at 2:00 p.m. PT / 5:00 p.m. ET today. The live webcast of Meta's earnings conference call can be accessed at investor.fb.com, along with the earnings press release, financial tables, and slide

In [10]:
document_path = Path("data/parsed_document.md")
with document_path.open("a") as f:
    f.write(parsed_doc.text)

#Step 4: Split and Embed Documents

In [15]:
import nltk

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [16]:
loader = UnstructuredMarkdownLoader(document_path)
loaded_documents = loader.load()

In [17]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2048, chunk_overlap=128)
docs = text_splitter.split_documents(loaded_documents)
len(docs)


11

In [18]:
print(docs[0].page_content)

Meta Reports First Quarter 2024 Results

MENLO PARK, Calif. – April 24, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter ended March 31, 2024.

"It's been a good start to the year," said Mark Zuckerberg, Meta founder and CEO. "The new version of Meta AI with Llama 3 is another step towards building the world's leading AI. We're seeing healthy growth across our apps and we continue making steady progress building the metaverse as well."

First Quarter 2024 Financial Highlights

In millions, except percentages and per share amounts 2024 2023 % Change Revenue $ 36,455 $ 28,645 27 % Costs and expenses $ 22,637 $ 21,418 6 % Income from operations $ 13,818 $ 7,227 91 % Operating margin 38 % 25 % Provision for income taxes $ 1,814 $ 1,598 14 % Effective tax rate 13 % 22 % Net income $ 12,369 $ 5,709 117 % Diluted earnings per share (EPS) $ 4.71 $ 2.20 114 %

First Quarter 2024 Operational and Other Financial Highlights

Family daily active people (DA

In [19]:
embeddings = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/740 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/218M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

In [20]:
qdrant = Qdrant.from_documents(
    docs,
    embeddings,
    # location=":memory:",
    path="./db",
    collection_name="document_embeddings",
)


# Perform Similarity Search

In [23]:
query = "What is the most important innovation from Meta?"
similar_docs = qdrant.similarity_search_with_score(query)

In [24]:
for doc, score in similar_docs:
    print(f"text: {doc.page_content[:256]}\n")
    print(f"score: {score}")
    print("-" * 80)
    print()

text: Meta Reports First Quarter 2024 Results

MENLO PARK, Calif. – April 24, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter ended March 31, 2024.

"It's been a good start to the year," said Mark Zuckerberg, Meta foun

score: 0.6171743278205013
--------------------------------------------------------------------------------

text: Webcast and Conference Call Information

Meta will host a conference call to discuss the results at 2:00 p.m. PT / 5:00 p.m. ET today. The live webcast of Meta's earnings conference call can be accessed at investor.fb.com, along with the earnings press rel

score: 0.5704010585179788
--------------------------------------------------------------------------------

text: Reconciliation of cash, cash equivalents, and restricted cash to the condensed consolidated balance sheets

Cash and cash equivalents $ 32,307 $ 11,551 Restricted cash, included in prepaid expenses and other current assets 84 224 Restricted cash, inc

In [25]:
retriever = qdrant.as_retriever(search_kwargs={"k": 5})
retrieved_docs = retriever.invoke(query)

for doc in retrieved_docs:
    print(f"id: {doc.metadata['_id']}\n")
    print(f"text: {doc.page_content[:256]}\n")
    print("-" * 80)
    print()

id: 996df08e54f74446b2f97486b63b2982

text: Meta Reports First Quarter 2024 Results

MENLO PARK, Calif. – April 24, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter ended March 31, 2024.

"It's been a good start to the year," said Mark Zuckerberg, Meta foun

--------------------------------------------------------------------------------

id: 493719b932aa4903a53f02fb61ef8ced

text: Webcast and Conference Call Information

Meta will host a conference call to discuss the results at 2:00 p.m. PT / 5:00 p.m. ET today. The live webcast of Meta's earnings conference call can be accessed at investor.fb.com, along with the earnings press rel

--------------------------------------------------------------------------------

id: 4aa97369a4a1463081c56196b5824e4c

text: Reconciliation of cash, cash equivalents, and restricted cash to the condensed consolidated balance sheets

Cash and cash equivalents $ 32,307 $ 11,551 Restricted cash, included in prepaid

#Contextual Compression

In [26]:
compressor = FlashrankRerank(model="ms-marco-MiniLM-L-12-v2")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

ms-marco-MiniLM-L-12-v2.zip: 100%|██████████| 21.6M/21.6M [00:00<00:00, 38.6MiB/s]


In [27]:
reranked_docs = compression_retriever.invoke(query)
len(reranked_docs)

Running pairwise ranking..


3

In [28]:
for doc in reranked_docs:
    print(f"id: {doc.metadata['_id']}\n")
    print(f"text: {doc.page_content[:256]}\n")
    print(f"score: {doc.metadata['relevance_score']}")
    print("-" * 80)
    print()

id: 996df08e54f74446b2f97486b63b2982

text: Meta Reports First Quarter 2024 Results

MENLO PARK, Calif. – April 24, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter ended March 31, 2024.

"It's been a good start to the year," said Mark Zuckerberg, Meta foun

score: 0.19008274376392365
--------------------------------------------------------------------------------

id: 493719b932aa4903a53f02fb61ef8ced

text: Webcast and Conference Call Information

Meta will host a conference call to discuss the results at 2:00 p.m. PT / 5:00 p.m. ET today. The live webcast of Meta's earnings conference call can be accessed at investor.fb.com, along with the earnings press rel

score: 0.007560899946838617
--------------------------------------------------------------------------------

id: 477ff3fe65174cc6820d4895f217937a

text: This press release contains forward-looking statements regarding our future business plans and expectations. These forward-looking st

#Question-Answering System

In [29]:
llm = ChatGroq(temperature=0, model_name="llama3-70b-8192")


prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Answer the question and provide additional helpful information,
based on the pieces of information, if applicable. Be succinct.

Responses should be properly formatted to be easily read.
"""

prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)


qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=compression_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt, "verbose": True},
)




In [30]:
def print_response(response):
    response_txt = response["result"]
    for chunk in response_txt.split("\n"):
        if not chunk:
            print()
            continue
        print("\n".join(textwrap.wrap(chunk, 100, break_long_words=False)))

In [31]:
response = qa.invoke("What is the most significant innovation from Meta?")

Running pairwise ranking..


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: Meta Reports First Quarter 2024 Results

MENLO PARK, Calif. – April 24, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter ended March 31, 2024.

"It's been a good start to the year," said Mark Zuckerberg, Meta founder and CEO. "The new version of Meta AI with Llama 3 is another step towards building the world's leading AI. We're seeing healthy growth across our apps and we continue making steady progress building the metaverse as well."

First Quarter 2024 Financial Highlights

In millions, except percentages and per share amounts 2024 2023 % Change Revenue $ 36,455 $ 28,645 27 % Costs and expenses $ 22,637 $ 21,418 6 % Income from operations 

In [32]:
print_response(response)

Based on the provided information, the most significant innovation from Meta is the new version of
Meta AI with Llama 3, which is mentioned in the quote from Mark Zuckerberg, Meta founder and CEO.
This innovation is part of Meta's efforts to build the world's leading AI.

Additionally, the press release highlights Meta's progress in building the metaverse, which is
another significant innovation from the company. However, the details of these innovations are not
elaborated upon in the provided information.


In [33]:
response = qa.invoke("What is the revenue for 2024 and % change?")

Running pairwise ranking..


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: (In millions, except percentages)

(Unaudited)

Three Months Ended March 31,

2024 2023 GAAP revenue $ 36,455 $ 28,645 Foreign exchange effect on 2024 revenue using 2023 rates (106) Revenue excluding foreign exchange effect $ 36,349 GAAP revenue year-over-year change % 27 % Revenue excluding foreign exchange effect year-over-year change % 27 % GAAP advertising revenue $ 35,635 $ 28,101 Foreign exchange effect on 2024 advertising revenue using 2023 rates (105) Advertising revenue excluding foreign exchange effect $ 35,530 GAAP advertising revenue year-over-year change % 27 % Advertising revenue excluding foreign exchange effect year-over-year change % 26 % Net cash provided by operatin

In [34]:
Markdown(response["result"])

**Revenue for 2024 and % Change:**

* Revenue for 2024: $36,455 million
* % Change: 27% year-over-year

**Additional Helpful Information:**

* Revenue excluding foreign exchange effect: $36,349 million
* Revenue excluding foreign exchange effect year-over-year change: 27%

Note: The revenue figures are based on the "Three Months Ended March 31" section of the provided text.

In [35]:
%%time
response = qa.invoke("What is the revenue for 2023?")

Running pairwise ranking..


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: (In millions, except percentages)

(Unaudited)

Three Months Ended March 31,

2024 2023 GAAP revenue $ 36,455 $ 28,645 Foreign exchange effect on 2024 revenue using 2023 rates (106) Revenue excluding foreign exchange effect $ 36,349 GAAP revenue year-over-year change % 27 % Revenue excluding foreign exchange effect year-over-year change % 27 % GAAP advertising revenue $ 35,635 $ 28,101 Foreign exchange effect on 2024 advertising revenue using 2023 rates (105) Advertising revenue excluding foreign exchange effect $ 35,530 GAAP advertising revenue year-over-year change % 27 % Advertising revenue excluding foreign exchange effect year-over-year change % 26 % Net cash provided by operatin

In [36]:
print_response(response)

**Answer:** The revenue for 2023 is $28,645 million.

**Additional helpful information:**

* The revenue for 2024 is $36,455 million, which is a 27% year-over-year increase.
* The revenue excluding foreign exchange effect for 2024 is $36,349 million, which is also a 27%
year-over-year increase.


In [37]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=compression_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt, "verbose": True},
)


In [38]:

%%time
response = qa.invoke(
    "How much is the revenue minus the costs and expenses for 2024? Calculate the answer"
)

Running pairwise ranking..


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: Headcount – Headcount was 69,329 as of March 31, 2024, a decrease of 10% year-over-year.

CFO Outlook Commentary

We expect second quarter 2024 total revenue to be in the range of $36.5-39 billion. Our guidance assumes foreign currency is a 1% headwind to year-over-year total revenue growth, based on current exchange rates.

We expect full-year 2024 total expenses to be in the range of $96-99 billion, updated from our prior outlook of $94-99 billion due to higher infrastructure and legal costs. For Reality Labs, we continue to expect operating losses to increase meaningfully year-over-year due to our ongoing product development efforts and our investments to further scale our ecosyste

In [39]:

print_response(response)


Based on the provided information, the revenue minus the costs and expenses for 2024 is:

Revenue: $36,455 million
Costs and expenses: $22,637 million

Revenue minus costs and expenses: $13,818 million

This is also referred to as the "Income from operations" in the financial highlights section.

Additionally, this amount represents a 91% year-over-year increase, and the operating margin is 38%.


In [40]:
%%time
response = qa.invoke(
    "How much is the revenue minus the costs and expenses for 2023? Calculate the answer"
)

Running pairwise ranking..


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: (In millions, except percentages)

(Unaudited)

Three Months Ended March 31,

2024 2023 GAAP revenue $ 36,455 $ 28,645 Foreign exchange effect on 2024 revenue using 2023 rates (106) Revenue excluding foreign exchange effect $ 36,349 GAAP revenue year-over-year change % 27 % Revenue excluding foreign exchange effect year-over-year change % 27 % GAAP advertising revenue $ 35,635 $ 28,101 Foreign exchange effect on 2024 advertising revenue using 2023 rates (105) Advertising revenue excluding foreign exchange effect $ 35,530 GAAP advertising revenue year-over-year change % 27 % Advertising revenue excluding foreign exchange effect year-over-year change % 26 % Net cash provided by operatin

In [41]:

print_response(response)

Based on the provided information, we can calculate the revenue minus the costs and expenses for
2023 as follows:

Revenue (2023) = $28,645
Income from operations (2023) = $7,227

Since income from operations is revenue minus costs and expenses, we can set up the equation:

Revenue - Costs and Expenses = Income from Operations
$28,645 - Costs and Expenses = $7,227

To find the costs and expenses, we can subtract the income from operations from the revenue:

Costs and Expenses = Revenue - Income from Operations
= $28,645 - $7,227
= $21,418

So, the revenue minus the costs and expenses for 2023 is $7,227.

Additional helpful information:

* The revenue for 2023 is broken down into two segments: Family of Apps (FoA) and Reality Labs (RL).
FoA revenue is $28,306, and RL revenue is $339.
* The income from operations for 2023 is also broken down into two segments: FoA income from
operations is $11,219, and RL income from operations is ($3,992), indicating a loss.


In [42]:
response = qa.invoke("What is the expected revenue for the second quarter of 2024?")

Running pairwise ranking..


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: Headcount – Headcount was 69,329 as of March 31, 2024, a decrease of 10% year-over-year.

CFO Outlook Commentary

We expect second quarter 2024 total revenue to be in the range of $36.5-39 billion. Our guidance assumes foreign currency is a 1% headwind to year-over-year total revenue growth, based on current exchange rates.

We expect full-year 2024 total expenses to be in the range of $96-99 billion, updated from our prior outlook of $94-99 billion due to higher infrastructure and legal costs. For Reality Labs, we continue to expect operating losses to increase meaningfully year-over-year due to our ongoing product development efforts and our investments to further scale our ecosyste

In [43]:
print_response(response)

**Answer:** The expected revenue for the second quarter of 2024 is in the range of $36.5-39 billion.

**Additional helpful information:** This guidance assumes a 1% headwind to year-over-year total
revenue growth due to foreign currency exchange rates.


In [44]:
response = qa.invoke("What is the overall outlook of Q1 2024?")

Running pairwise ranking..


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: Headcount – Headcount was 69,329 as of March 31, 2024, a decrease of 10% year-over-year.

CFO Outlook Commentary

We expect second quarter 2024 total revenue to be in the range of $36.5-39 billion. Our guidance assumes foreign currency is a 1% headwind to year-over-year total revenue growth, based on current exchange rates.

We expect full-year 2024 total expenses to be in the range of $96-99 billion, updated from our prior outlook of $94-99 billion due to higher infrastructure and legal costs. For Reality Labs, we continue to expect operating losses to increase meaningfully year-over-year due to our ongoing product development efforts and our investments to further scale our ecosyste

In [45]:

print_response(response)


**Overall Outlook of Q1 2024:**

The overall outlook of Q1 2024 is positive. According to Mark Zuckerberg, Meta's founder and CEO,
"It's been a good start to the year." The company has reported strong momentum within its Family of
Apps and made important progress on its longer-term AI and Reality Labs initiatives.

**Additional Helpful Information:**

* Revenue increased by 27% year-over-year to $36.46 billion.
* Net income increased by 117% year-over-year to $12.37 billion.
* Diluted earnings per share (EPS) increased by 114% year-over-year to $4.71.
* Family daily active people (DAP) increased by 7% year-over-year to 3.24 billion.
* Ad impressions increased by 20% year-over-year.
* Average price per ad increased by 6% year-over-year.
